# Module 1: Multiple Sequence Alignment (MSA)

In this practical you will:
- Generate multiple sequence alignments using different methods
- Compare alignments visually and quantitatively
- Decide which alignment is most appropriate for downstream phylogenetics

This notebook contains:
- **Worked examples**
- **Exercises where you will write code**
- **Optional extensions** for those who finish early

⚠️ The *minimum deliverable* at the end of today is:
- One curated alignment (FASTA) that you are confident using for phylogenetic tree inference in Module 2.

## Explore the data

In [ ]:
# Run this to check your environment is set up correctly
# It should print the path to the python executable in your mol_phy_course environment

import sys
print(sys.executable)

In [ ]:
# Import packages

from Bio import SeqIO
from Bio.Seq import Seq
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

### Data

The `data/` directory contains a small example datasets that can be used throughout this course. Students may optionally replace this with their own data.

We will load a dataset of 372 rabies virus (RABV) sequences sampled from 17 bat species, taken between 1997 and 2006 across 14 states in the United States (originally from: https://beast.community/workshop_discrete_diffusion). After loading, we can get some basic understanding of the sequences

In [ ]:
# Load sequences
fasta_file = Path("../data/batRABV_raw_sequences.fas")
raw_seqs = list(SeqIO.parse(fasta_file, "fasta"))

# Basic stats
num_seqs = len(raw_seqs)
seq_lengths = [len(rec.seq) for rec in raw_seqs]
avg_length = sum(seq_lengths) / num_seqs

# Function to calculate GC content
def gc_content(seq):
    seq = seq.upper()
    gc = seq.count("G") + seq.count("C")
    return gc / len(seq) * 100 if len(seq) > 0 else 0

gc_contents = [gc_content(rec.seq) for rec in raw_seqs]
avg_gc = sum(gc_contents) / num_seqs

# Output
print(f"Number of sequences: {num_seqs}")
print(f"Sequence lengths: min={min(seq_lengths)}, max={max(seq_lengths)}, avg={avg_length:.1f}")
print(f"GC content (%): min={min(gc_contents):.1f}, max={max(gc_contents):.1f}, avg={avg_gc:.1f}")

# Show first few sequences as example
print("\nFirst 3 sequences:")
for rec in raw_seqs[:3]:
    print(f">{rec.id}\n{rec.seq}\n")

### Reading frame

For alignment and downstream analysis, it is important that alignments respect reading frame. To achieve this, all nucleotide sequences must have length that is a multiple of 3. We check this for the sequences below. It is also good practice to have sequences that start with the start codon ATG. This can be useful information if we need to trim sequences. For our purposes and for our dataset, we disregard that some sequences do not strictly begin with ATG.

In [ ]:
# Check if sequences are divisible by 3
not_div3 = [rec.id for rec in raw_seqs if len(rec.seq) % 3 != 0]

if not_div3:
    print("\n⚠️ Warning: some sequences are not divisible by 3 (may not be in-frame)")
else:
    print("\nAll sequences have lengths divisible by 3.")

# Check if sequences start with ATG
not_start_atg = [rec.id for rec in raw_seqs if not rec.seq.upper().startswith("ATG")]

if not_start_atg:
    print("\n⚠️ Warning: some sequences do not start with ATG (may not be valid CDS)")
else:
    print("\nAll sequences start with ATG.")



## Task 1: Alignment

We will use two different, widely-used aligners for comparison purposes: Clustal and MAFFT. There are different ways to access these alignment tools:

1. **Web services (recommended)**
This is the easiest option and viable when working with a manageable number of sequences, nucleotides and MSAs.
Upload the sequences `batRABV_raw_sequences.fas` directly:
- Clustal: https://www.ebi.ac.uk/jdispatcher/msa/clustalo
- MAFFT: https://www.ebi.ac.uk/jdispatcher/msa/mafft/
Then download the aligned sequences. 

2. **Download and compile alignment tools from source**
If you are familar with compiling tools, you can find the download links here:
- Clustal https://bio.tools/clustalw
- MAFFT: https://mafft.cbrc.jp/alignment/software/
Clustal and MAFFT are amino acid (AA) aligners, so require the sequences to be in AA form rather than nucleotide. You can convert the sequences to AA using a standard package or write your own code to do it, then convert back after alignment. 

3. **Install packages to conda environment**
This used to be a great solution, however, Clustal and MAFFT are only availble on conda through the `bioconda` channel which is now behind a paywall that EMBL hasn't subscribed to. So if you are able to use the `bioconda` channel and want to do it a more 'programmatic' way, then go for it. As with option 2, you will first need to convert the nucleotide sequences to AA sequences for alignment, and then convert back to nucleotide after alignment.  



## Task 2

PRANK is a highly accurate alignment tool that creates a phylogenetic tree to guide the alignment process. As a result, it is slower than other algorithms. Use PRANK to align the sequences are compare the resulting MSA to those you created using Clustal and MAFFT. PRANK is also available as a web service (https://www.ebi.ac.uk/goldman-srv/webprank/) or can be compiled from source. 